In [43]:
from sklearn.svm import SVR
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import math
import pandas as pd 
import numpy as np 
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
import warnings
pd.set_option('mode.chained_assignment', None)
warnings.simplefilter(action='ignore', category=FutureWarning)
import tensorflow as tf
import keras 
from keras.models import Sequential 
from keras.layers import Dense, LSTM
plt.style.use('fivethirtyeight')
df = pd.read_csv('XAU5Y.csv')
df2 = pd.read_csv('USGGT10Y.csv')
data2 = df2.filter(['Last Price'])
data = df.filter(['Close'])
df

,Date,Open,High,Low,Close
0,7/8/2016,1360.45,1370.39,1335.47,1366.33
1,7/11/2016,1369.70,1375.34,1350.90,1355.40
2,7/12/2016,1355.38,1357.86,1330.41,1333.10
3,7/13/2016,1333.05,1345.41,1327.61,1342.64
4,7/14/2016,1342.65,1347.27,1320.35,1335.23
...,...,...,...,...,...
1266,5/24/2021,1882.03,1887.07,1875.25,1881.02
1267,5/25/2021,1881.02,1900.29,1872.78,1899.25
1268,5/26/2021,1899.25,1912.76,1890.78,1896.67
1269,5/27/2021,1896.68,1903.70,1888.34,1896.54


In [ ]:
#seeing the data 
plt.figure(figsize=(16,6))
plt.title('close price')
plt.plot(df['Close'])
plt.xlabel('Date',fontsize=18)
plt.ylabel('Close price USD', fontsize=18)
plt.show()

In [ ]:
close_px = df['Close']
mavg = close_px.rolling(window=100).mean()

plt.figure(figsize = (12,6))
close_px.plot(label='XAU')
mavg.plot(label='mavg')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()

last_px = df2['Last Price']
mavg = last_px.rolling(window=100).mean()

plt.figure(figsize = (12,6))
last_px.plot(label='Real Rates')
mavg.plot(label='mavg')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()

In [ ]:
#create data frame with close 
data2 = data.join(data2, how = 'inner')
dataset = data2.values
#getting rows 
training_data_length = math.ceil(len(dataset)*.8)
training_data_length
#scaling 
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)
#training_data_length, scaled_data
training_data_length

In [ ]:
dates = df.filter(['Date'])
df['Date'] = pd.to_datetime(df['Date'])
df_reshape = df.filter(['Close'])

dates = np.reshape(dates, (len(dates), 1))
prices = np.reshape(df_reshape, (len(df_reshape), 1))

svr_rbf = SVR(kernel= 'rbf', C= 1e3, gamma= 0.1)
svr_rbf.fit(dates, df_reshape)

plt.figure(figsize = (12,6))
plt.plot(dates, prices, color= 'black', label= 'Data')
plt.plot("Date", svr_rbf.predict(dates), color= 'red', label= 'RBF model') 
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.show()

In [ ]:
#create training dataset for scaled data 
train_data = scaled_data[0:training_data_length, :]
x_train = []
y_train = []
for i in range(120, len(train_data)):
    x_train.append(train_data[i-120:(i), :])
    y_train.append(train_data[i,:])
    if i<= 121:
        print(x_train)
        print(y_train)
        print()
#making trainable arrays 
x_train, y_train = np.array(x_train), np.array(y_train)
x_train= np.reshape(x_train, (x_train.shape[0],x_train.shape[1],2))
y_train.shape

In [50]:
#build model
model = Sequential()
model.add(LSTM(32, return_sequences=True, input_shape = (x_train.shape[1],2)))
model.add(LSTM(32,return_sequences = True))
model.add(LSTM(32, return_sequences= False))
model.add(Dense(units=2))
model.add(Dense(2))
model.summary()
model.compile(optimizer='adam', loss='mean_squared_error')



#train model 
model.fit(x_train, y_train, batch_size=2, epochs=1,validation_data=(x_test, y_test), verbose=2, shuffle=False)

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_16 (LSTM)               (None, 120, 32)           4480      
_________________________________________________________________
lstm_17 (LSTM)               (None, 120, 32)           8320      
_________________________________________________________________
lstm_18 (LSTM)               (None, 32)                8320      
_________________________________________________________________
dense_9 (Dense)              (None, 2)                 66        
_________________________________________________________________
dense_10 (Dense)             (None, 2)                 6         
Total params: 21,192
Trainable params: 21,192
Non-trainable params: 0
_________________________________________________________________
Train on 860 samples, validate on 244 samples
Epoch 1/1
 - 34s - loss: 0.0084 - val_loss: 1685448.4436


In [51]:
#testing dataset 
#897 to 1271
test_data = scaled_data[training_data_length - 120: , :]
x_test = []
y_test= dataset[training_data_length:]
for i in range(120, len(test_data)):
    x_test.append(test_data[i-120:(i), :])
#back to numpy array 
x_test = np.array(x_test)
#make data 3d 
x_test = np.reshape(x_test,(x_test.shape[0], x_test.shape[1], 2))
x_test.shape

(244, 120, 2)

In [52]:
#prediction
#predictions = model.predict(x_test)
#trainPredict_extented = np.zeros((len(predictions),2))
#trainPredict_extented[: , :] = predictions
#predictions = scaler.inverse_transform(predictions)
#rmse=np.sqrt(np.mean(((predictions - y_test)**2)))


predictions = model.predict(x_test)
x_test = x_test.reshape((x_test.shape[0],x_test.shape[2]))
invert_predictions = concatenate((predictions, x_test[:, 1:]), axis=1)
invert_predictions = scaler.inverse_transform(invert_predictions)
invert_predictions = invert_predictions[:,0]
y_test = y_test.reshape((len(y_test), 1))
invert_y = concatenate((y_test,x_test[:,1:]), axis=1)
invert_y = scaler.inverse_transform(invert_y)
invert_y = invert_y[:,0]
#plots 
train = data2[:training_data_length]
validation = data2[training_data_length:]
validation['Predictions', 'Predictions1'] = predictions
plt.figure(figsize=(14,6))
plt.title('Model')
plt.xlabel('Date',fontsize=18)
plt.ylabel('Close Price', fontsize=18)
plt.plot(train['Close'])
plt.plot(validation[['Close','Predictions']])
plt.legend(['Train', 'Validations', 'Predictions'], loc='lower right')
plt.show(), validation
predictions

ValueError: cannot reshape array of size 58560 into shape (244,2)

In [53]:
#attempting to predict the closing price for 5/31/2021
new_df = df.filter(['Close'])
last_120_days = new_df[-120:].values 
last_120_days_scaled = scaler.transform(last_120_days)
X_test =[]
X_test.append(last_120_days_scaled)
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1],1))
pred_price = model.predict(X_test)
pred_price = scaler.inverse_transform(pred_price)
print(pred_price)

ValueError: non-broadcastable output operand with shape (120,1) doesn't match the broadcast shape (120,2)